In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
# size of the input at each time step
# TODO: input sizes????
input_size = 5
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 8]
# TODO: ouptut sizes????
output_size = 10
# number of LSTM layers
n_layers = 1

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, ouput_size, n_layers, hidden_size):
        super(NeuralNetwork, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size[0], n_layers, batch_first=True)
        
        # First fully connected layer
        self.fc1 = nn.Linear(hidden_size[1], output_size)
        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_size[2], output_size)
        
        # Softmax activation
        self.softmax = nn.Softmax()
        
    def forward(self, x, hidden):
        # x(batch_size, seq_length, input_size)
        # hidden(n_layers, batch_size, hidden_dim)
        # r_out(batch_size, time_step, hidden_size)
        batch_size = x.size(0)
        
        # get LSTM outputs
        lstm_out, hidden = self.lstm(x, hidden)
        
        # shape output to be (batch_size*seq_length, hidden_dim)
        lstm_out = lstm_out.view(-1, self.hidden_dim)
        
        # get output of first fully connected layer
        out1 = self.fc1(lstm_out)
        
        # get output of second fully connected layer
        out2 = self.fc2(out1)
        
        # softmax activation function
        out2 = self.softmax(out2)
        
        # TODO: might need to reshape the out2
        
        return output, hidden